In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

sms_spam = \
pd.read_csv('SMSSpamCollection', sep='\t', header=None, \
            names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

In [3]:
# labeling spam mail as 0;  ham mail as 1;

sms_spam.loc[sms_spam['Label'] == 'spam', 'Label'] = 1
sms_spam.loc[sms_spam['Label'] == 'ham', 'Label'] = 0

sms_spam

,Label,SMS
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ü b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [4]:
sms_spam["SMS_Copy"] = sms_spam["SMS"]
sms_spam.head()

,Label,SMS,SMS_Copy
0,0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro..."


In [5]:
import nltk

from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

def stem(txt):
    lst = []
    
    for i in txt.split():
        lst.append(ps.stem(i))
        
    return " ".join(lst)

sms_spam["SMS"] = sms_spam["SMS"].apply(stem)

In [6]:
sms_spam

,Label,SMS,SMS_Copy
0,0,"go until jurong point, crazy.. avail onli in b...","Go until jurong point, crazy.. Available only ..."
1,0,ok lar... joke wif u oni...,Ok lar... Joking wif u oni...
2,1,free entri in 2 a wkli comp to win fa cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,u dun say so earli hor... u c alreadi then say...,U dun say so early hor... U c already then say...
4,0,"nah i don't think he goe to usf, he live aroun...","Nah I don't think he goes to usf, he lives aro..."
...,...,...,...
5567,1,thi is the 2nd time we have tri 2 contact u. u...,This is the 2nd time we have tried 2 contact u...
5568,0,will ü b go to esplanad fr home?,Will ü b going to esplanade fr home?
5569,0,"pity, * wa in mood for that. so...ani other su...","Pity, * was in mood for that. So...any other s..."
5570,0,the guy did some bitch but i act like i'd be i...,The guy did some bitching but I acted like i'd...


In [7]:
# separating the data as texts and label

X = sms_spam['SMS']

Y = sms_spam['Label']

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=5)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# Transforming the text data to bag-of-words feature vectors
feature_extraction = CountVectorizer(stop_words='english')

X_train_features_bow = feature_extraction.fit_transform(X_train)
X_test_features_bow = feature_extraction.transform(X_test)

# Converting Y_train and Y_test values to integers
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [10]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(5014,)
(558,)


In [11]:
X_train_features_bow.toarray().shape

(5014, 7554)

In [12]:
X_train_features_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
model = LogisticRegression()

In [14]:
# training the Logistic Regression model with the training data
model.fit(X_train_features_bow, Y_train)

LogisticRegression()

Evaluating the trained model

In [15]:
# prediction on training data

prediction_on_training_data = model.predict(X_train_features_bow)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [16]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.9962106102911846


In [17]:
# prediction on testing data

prediction_on_testing_data = model.predict(X_test_features_bow)
accuracy_on_testing_data = accuracy_score(Y_test, prediction_on_testing_data)
print('Accuracy on training data : ', accuracy_on_testing_data)

Accuracy on training data :  0.9838709677419355


In [18]:
sms_spam[sms_spam['Label']==1]

,Label,SMS,SMS_Copy
2,1,free entri in 2 a wkli comp to win fa cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
5,1,freemsg hey there darl it' been 3 week' now an...,FreeMsg Hey there darling it's been 3 week's n...
8,1,winner!! as a valu network custom you have bee...,WINNER!! As a valued network customer you have...
9,1,had your mobil 11 month or more? u r entitl to...,Had your mobile 11 months or more? U R entitle...
11,1,"six chanc to win cash! from 100 to 20,000 poun...","SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...,...
5537,1,want explicit sex in 30 secs? ring 02073162414...,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ask 3mobil if 0870 chatlin inclu in free mins....,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,had your contract mobil 11 mnths? latest motor...,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,remind from o2: to get 2.50 pound free call cr...,REMINDER FROM O2: To get 2.50 pounds free call...


In [19]:
def sms(input_mail):
    input_data_features = feature_extraction.transform([input_mail])
    prediction = model.predict(input_data_features)
    print(prediction)

    if (prediction==1):
        print('Spam SMS')
    else:
        print('Ham SMS')

In [20]:
sms("Congratulations! You've won a $1000 cash prize")

[1]
Spam SMS


In [21]:
spam_only = sms_spam[sms_spam["Label"] == 1].reset_index(drop =True)
spam_only.iloc[2][2]

'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

In [22]:
cv = CountVectorizer(stop_words='english')

In [23]:
vector = cv.fit_transform(spam_only['SMS']).toarray()
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
similarity = cosine_similarity(vector)

In [26]:
similarity

array([[1.        , 0.05063697, 0.        , ..., 0.1315587 , 0.10127394,
        0.04499213],
       [0.05063697, 1.        , 0.        , ..., 0.        , 0.06666667,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.05923489,
        0.15789474],
       ...,
       [0.1315587 , 0.        , 0.        , ..., 1.        , 0.17320508,
        0.        ],
       [0.10127394, 0.06666667, 0.05923489, ..., 0.17320508, 1.        ,
        0.11846978],
       [0.04499213, 0.        , 0.15789474, ..., 0.        , 0.11846978,
        1.        ]])

In [27]:
def type_of_msgs(msg):
    input_data_features = feature_extraction.transform([msg])
    prediction = model.predict(input_data_features)
    if (prediction[0]==1):
        query_vec = cv.transform([msg])
        word_similarity = cosine_similarity(query_vec, vector)[0]
        most_similar = sorted(list(enumerate(word_similarity)), reverse=True, key = lambda x:x[-1])[:10]
        print('Spam SMS')
        for i in most_similar:
            print(spam_only.iloc[i[0],2])
    else:
      print('Ham SMS')
    

In [28]:
type_of_msgs("Congratulations! You've won a $1000 cash prize. Claim your prize now by clicking the link: http://example.com/claim")

Spam SMS
You have WON a guaranteed £1000 cash or a £2000 prize.To claim yr prize call our customer service representative on
You have WON a guaranteed £1000 cash or a £2000 prize. To claim yr prize call our customer service representative on 08714712394 between 10am-7pm
You have WON a guaranteed £1000 cash or a £2000 prize. To claim yr prize call our customer service representative on 08714712379 between 10am-7pm Cost 10p
You have WON a guaranteed £1000 cash or a £2000 prize. To claim yr prize call our customer service representative on 08714712412 between 10am-7pm Cost 10p
Win a £1000 cash prize or a prize worth £5000
You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327
You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327. T&C: RSTM, SW7 3SS. 150ppm
Congratulations YOU'VE Won. You're a Winner in our August £1000 Prize Draw. Call 09066660100 NOW. Prize Code 2309.
This is the 2nd attempt to contract U, you have won this weeks top prize of either £1000 c